<a href="https://colab.research.google.com/github/sorrawich-chai/qmsum_on_longformer/blob/main/test_fined_tune_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 72.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 78.8 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline

model_checkpoint = "fgiuhsdfkjhfv/longsec3"
sum = pipeline("summarization", model=model_checkpoint,device=0)

In [ ]:
!pip install nltk
!git clone https://github.com/Yale-LILY/QMSum.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Cloning into 'QMSum'...
remote: Enumerating objects: 806, done.
remote: Counting objects: 100% (2/2), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 806 (delta 1), reused 0 (delta 0), pack-reused 804
Receiving objects: 100% (806/806), 13.76 MiB | 16.71 MiB/s, done.
Resolving deltas: 100% (446/446), done.


In [ ]:
import json

# read the dataset
# please enter the path of your data
split = 'test'
data_path = 'QMSum/data/ALL/jsonl/' + split + '.jsonl'
data = []
with open(data_path) as f:
    for line in f:
        data.append(json.loads(line))
n_meetings = len(data)
print('Total {} meetings in the {} set.'.format(n_meetings, split))

Total 35 meetings in the test set.


In [ ]:
from nltk import word_tokenize
# tokneize a sent
def tokenize(sent):
    tokens = ' '.join(word_tokenize(sent.lower()))
    return tokens

In [ ]:
def clean_data(text):
    text = text.replace('{ vocalsound } ', '')
    text = text.replace('{ disfmarker } ', '')
    text = text.replace('a_m_i_', 'ami')
    text = text.replace('l_c_d_', 'lcd')
    text = text.replace('p_m_s', 'pms')
    text = text.replace('t_v_', 'tv')
    text = text.replace('{ pause } ', '')
    text = text.replace('{ nonvocalsound } ', '')
    text = text.replace('{ gap } ', '')
    return text

In [ ]:
import nltk
nltk.download('punkt')

# process data for BART
# the input of the model here is the gold span corresponding to each query
bart_data_gold = []
for i in range(len(data)):
    # get meeting content
    entire_src = []
    for k in range(len(data[i]['meeting_transcripts'])):
        cur_turn = data[i]['meeting_transcripts'][k]['speaker'].lower() + ': '
        cur_turn = cur_turn + tokenize(data[i]['meeting_transcripts'][k]['content'])
        entire_src.append(cur_turn)
    entire_src = ' '.join(entire_src)
    for j in range(len(data[i]['general_query_list'])):
        cur = {}
        query = tokenize(data[i]['general_query_list'][j]['query'])
        cur['src'] = clean_data('<s> ' + query + ' </s> ' + entire_src + ' </s>')
        target = tokenize(data[i]['general_query_list'][j]['answer'])
        cur['tgt'] = target
        bart_data_gold.append(cur)
    for j in range(len(data[i]['specific_query_list'])):
        cur = {}
        query = tokenize(data[i]['specific_query_list'][j]['query'])
        src = []
        # get the content in the gold span for each query
        for span in data[i]['specific_query_list'][j]['relevant_text_span']:
            assert len(span) == 2
            st, ed = int(span[0]), int(span[1])
            for k in range(st, ed + 1):
                cur_turn = data[i]['meeting_transcripts'][k]['speaker'].lower() + ': '
                cur_turn = cur_turn + tokenize(data[i]['meeting_transcripts'][k]['content'])
                src.append(cur_turn)
        src = ' '.join(src)
        cur['src'] = clean_data('<s> ' + query + ' </s> ' + src + ' </s>')
        target = tokenize(data[i]['specific_query_list'][j]['answer'])
        cur['tgt'] = target
        bart_data_gold.append(cur)
        
print('Total {} query-summary pairs in the {} set'.format(len(bart_data_gold), split))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Total 281 query-summary pairs in the test set


In [ ]:
bart_data_gold[10]['tgt']

'barry hughes explained a number of unintended consequences questioned . the first one was about the isolated incidences in wales , and the second came to the targets in terms of performance measure . the following concerns included malicious reporting , the european convention on human rights , low numbers of prosecutions , and a shortage of registered intermediaries .'

In [ ]:
predict = sum(bart_data_gold[10]['src'])

In [ ]:
predict

[{'summary_text': "user interface thought that the remote control would be made of the team was to use , and they should be a more . it was not make the group on the project manager suggested that there could be used for the government had been in the meeting with the product as well about the user interface designer-s were also by this design 'sirsty will be some from the new button at the buttons ."}]

In [ ]:
!pip install datasets
!pip install sacrebleu
!pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 5.3 MB/s eta 0:00:00


In [ ]:
from datasets import load_metric

metric = load_metric("sacrebleu")
rouge = evaluate.load('rouge')

In [ ]:

predictions = [
   predict
]
references = [
    [
        bart_data_gold[4]['tgt']
    ]
]
rouge.compute(predictions=predictions, references=references)

{'score': 3.189422671511411,
 'counts': [33, 5, 1, 0],
 'totals': [96, 95, 94, 93],
 'precisions': [34.375,
  5.2631578947368425,
  1.0638297872340425,
  0.5376344086021505],
 'bp': 1.0,
 'sys_len': 96,
 'ref_len': 77}